# Exploratory Analysis
---

This code imports zipped data files from an S3 bucket, determines the shape and size of the data frame (Pandas), detects outliers/NAN/Null values, calculates collumn-wise mean/median/mode/stddev, and performs some correlation analysis on the variables.

# Dependencies & Functions
---

In [ ]:
import pandas as pd
import boto3
import botocore
import sagemaker
import numpy as np

#this is a test
def eda(df):
    # Source: https://gist.github.com/jiahao87/c97214065f996b76ab8fe4ca1964b2b5
    
    """Given dataframe, generate exploratory data analysis"""
    # check that input is pandas dataframe
    if type(df) != pd.core.frame.DataFrame:
        raise TypeError("Only pandas dataframe is allowed as input")
        
    # replace field that's entirely space (or empty) with NaN
    df = df.replace(r'^\s*$', np.nan, regex=True)

    print("Preview of data:")
    display(df.head(3))

    print("\nTo check: \n (1) Total number of entries \n (2) Column types \n (3) Any null values\n")
    print(df.info())

    # generate preview of entries with null values
    if df.isnull().any(axis=None):
        print("\nPreview of data with null values:")
        display(df[df.isnull().any(axis=1)].head(3))
        missingno.matrix(df)
        plt.show()

    # generate count statistics of duplicate entries
    if len(df[df.duplicated()]) > 0:
        print("\n***Number of duplicated entries: ", len(df[df.duplicated()]))
        display(df[df.duplicated(keep=False)].sort_values(by=list(df.columns)).head())
    else:
        print("\nNo duplicated entries found")

# Import Data
---

In [20]:
# Import dataset from S3 Bucket

bucket = "exploratoryanalysis-swat"             # Bucket location for working directory
prefix = "analysis_artifacts"                   # Folder location for working directory

# S3 bucket where the original data is downloaded and stored.
downloaded_data_bucket = "exploratoryanalysis-swat"    # Bucket location for data directory
downloaded_data_prefix = "dataset_unzipped"            # Folder location of data files

# Housekeeping, sagemaker gets execution role and reigon from this sagemaker instance
execution_role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# Check bucket existence and permissions
def check_bucket_permission(bucket):
    # check if the bucket exists
    permission = False
    try:
        boto3.Session().client("s3").head_bucket(Bucket=bucket)
    except botocore.exceptions.ParamValidationError as e:
        print(
            "Hey! You either forgot to specify your S3 bucket"
            " or you gave your bucket an invalid name!"
        )
    except botocore.exceptions.ClientError as e:
        if e.response["Error"]["Code"] == "403":
            print(f"Hey! You don't have permission to access the bucket, {bucket}.")
        elif e.response["Error"]["Code"] == "404":
            print(f"Hey! Your bucket, {bucket}, doesn't exist!")
        else:
            raise
    else:
        permission = True
    return permission


if check_bucket_permission(bucket):
    print(f"Training input/output will be stored in: s3://{bucket}/{prefix}")
if check_bucket_permission(downloaded_data_bucket):
    print(f"Downloaded training data will be read from s3://{downloaded_data_bucket}/{downloaded_data_prefix}")
# Future development: expand for more data upload options, perhaps a filepath instead

Training input/output will be stored in: s3://exploratoryanalysis-swat/analysis_artifacts
Downloaded training data will be read from s3://exploratoryanalysis-swat/dataset_unzipped


# Classify & Organize
---

In [ ]:
# Organize and qualify data

# Input file name here
data_filename = "SWaT_Dataset_Normal_v1_01.csv"

# Download S3 data file to Sagemaker space
s3 = boto3.client("s3")
s3.download_file(downloaded_data_bucket, f"{downloaded_data_prefix}/{data_filename}", data_filename)

# Read file into Pandas Dataframe 
sensor_data = pd.read_csv(data_filename)

# Establish Shape of raw sensor data
raw_data_shape = sensor_data.shape
print("Data file dimensions:", raw_data_shape)

In [ ]:
eda(sensor_data)

# Clean Data
---

In [ ]:
# Locate NAN values and store indicies
nan_values = sensor_data[sensor_data.isna().any(axis=1)]
if nan_values.empty:
    pass
else:
    # Determine what we want to do with the missing fields
    print('There are NAN values in the data')
    
    
# I already know that there is a misspelling of the word "Attack" that reads "A ttack" which will need to be cleaned in the attack version of the dataset.
# This cleaning has been performed within the .csv direclty

# I also see another issue where the SWaT Attack dataset does not have the P1, P2, ... P5 headers above the individual collumns
# This has been corrected by removing the process headers

# Analyze Data
---

In [ ]:
# Determine mean, median, std dev, etc. of each collumn
data_statistics = sensor_data.describe()
print("Dataset statistics for each column:")
pd.set_option('display.max_columns', None)
display(data_statistics)

#print(data_statistics, '\n')

In [ ]:
# What are the unique values and counts for each categorical column
unique_values = sensor_data.nunique(axis=0)
print("Number of unique values for each column in dataset:")
display(unique_values)

# Which columns have only a single unique value? (Constants)
print("\nColumns with constant values:")
constant_columns = unique_values.where(unique_values==1).dropna(how='all')
print(constant_columns)

# Outliers
Z Score tells us how many standard deviations each value is from the mean. Anything outside of 3 STD deviations is an outlier and anything outside 2 is in the farthest 5% and could be cut <br>
IQR determines the range between the 25th and 75th percentile <br>
Anything that lies 1.5*(range) above or below these quartiles is considered an outlier <br>

In [ ]:
# Locate and classify outliers
import numpy as np
from scipy import stats

# The collumns include two objects: a timestamp and a string. These cannot be analyzed for outliers, and will be removed by selecting only columns with numbers
numerical_sensor_data = sensor_data.select_dtypes(include='number')

# Z Score tells us how many standard deviations each value is from the mean. Anything outside of 3 STD deviations is an outlier and anything outside 2 is in the farthest 5% and could be cut
z_scores = numerical_sensor_data.apply(stats.zscore)

pd.set_option('display.max_columns', None)
display(z_scores.head())

In [ ]:
# Interquartile Range (IQR) is another way to classify outliers. 
# IQR determines the range between the 25th and 75th percentile
# Then, anything that lies 1.5*(range) above or below these quartiles is considered an outlier
# If the data value is lower than the lower range or larger than the upper range, store the index of the value.
# Then count the number of outliers

IQR = data_statistics.loc['75%'] - data_statistics.loc['25%']
IQR = IQR.to_frame()
IQR = IQR.transpose()

# Define lower and upper ranges for each variable. Statistics does not run on timestamp or string objects. Only columns with numerical values.
lower_range = data_statistics.loc['25%'] - 1.5 * IQR
upper_range = data_statistics.loc['75%'] + 1.5 * IQR

# The format of a query needs to be "column label <query condition> @<variable name to be compared against>"
# 'FIT101<@aa' where aa is the variable that holds the limit value for this iteration
lower_queries = ['{}<@aa'.format(k) for k in lower_range.columns]
upper_queries = ['{}>@aa'.format(k) for k in upper_range.columns]

jj = 0
tot_upper_outl = 0
tot_lower_outl = 0
for column in IQR:
    # Lower boundary check
    aa = lower_range.iat[0,jj]                                        # ex: FIT101
    lower_outliers = numerical_sensor_data.query(lower_queries[jj])   # ex: 'FIT101<@aa'
    if not lower_outliers.empty:
        print('Condition that triggered the IQR outlier: ', lower_queries[jj], aa)
        print('Number of "less than" outliers', lower_outliers.shape[0], '\n')
        tot_lower_outl += lower_outliers.shape[0]
    
    # Upper Boundary Check
    aa = upper_range.iat[0,jj]                                        # ex: FIT101
    upper_outliers = numerical_sensor_data.query(upper_queries[jj])   # ex: 'FIT101>@aa'
    if not upper_outliers.empty:
        print('Condition that triggered the IQR outlier: ', upper_queries[jj], aa)
        print('Number of "greater than" outliers', upper_outliers.shape[0], '\n')
        tot_upper_outl += upper_outliers.shape[0]
    jj += 1
    
print('Total number of outliers: ', tot_lower_outl+tot_upper_outl)

In [35]:
# Correlation analysis
# Are there any correlations between the different columns in the data?
# Display correlation matrix

# First, drop all the columns with constant values. These will not impact a correlation matrix.
variable_sensor_data = sensor_data.drop(constant_columns.index, axis=1)

correlation_matrix = variable_sensor_data.corr()
pd.set_option('display.max_columns', None)
display(correlation_matrix.head())

# Threshold value (c
threshold_upper = 0.5
threshold_lower = -0.5
threshold_corr_matrix = correlation_matrix[(correlation_matrix > threshold_upper)]

# Upload correlation matrix CSV to S3 Artifacts
filename = 'correlation_matrix.csv'
correlation_matrix.to_csv(filename)

s3.upload_file(
    Filename=filename,
    Bucket=bucket,
    Key=f"{prefix}/{filename}"
)

,FIT101,LIT101,MV101,P101,AIT201,AIT202,AIT203,FIT201,MV201,P203,P205,DPIT301,FIT301,LIT301,MV301,MV302,MV303,MV304,P301,P302,AIT401,AIT402,FIT401,LIT401,P402,UV401,AIT501,AIT502,AIT503,AIT504,FIT501,FIT502,FIT503,FIT504,P501,PIT501,PIT502,PIT503,FIT601,P602
FIT101,1.000000,-0.610349,0.972333,0.256764,0.015445,-0.064270,0.052884,0.263456,0.257377,0.264456,0.133282,-0.209032,-0.218082,0.233469,0.028093,-0.191265,0.068349,0.003926,0.057647,-0.238541,-0.045521,0.154326,0.031704,0.269263,0.028480,0.028352,-0.055396,0.186126,0.113274,-0.019039,0.031871,0.033896,0.025366,0.018301,0.025039,0.032426,0.068567,0.033747,0.054980,0.051000
LIT101,-0.610349,1.000000,-0.609546,-0.606593,-0.050169,0.130691,-0.144169,-0.614798,-0.608094,-0.613915,-0.289995,-0.143930,-0.138820,0.353102,-0.030333,-0.129274,-0.073739,-0.010313,-0.220905,-0.064634,0.103748,-0.332853,0.257520,0.139218,0.252718,0.252883,0.444454,-0.438977,-0.092055,-0.260089,0.257721,0.254285,0.257933,0.259692,0.256979,0.253455,0.010261,0.252869,-0.059752,-0.055444
MV101,0.972333,-0.609546,1.000000,0.266683,0.015493,-0.065378,0.053816,0.273287,0.267042,0.274082,0.140212,-0.183593,-0.191641,0.219957,0.028272,-0.167931,0.068273,0.012059,0.057258,-0.213618,-0.045098,0.153793,0.030835,0.268360,0.027707,0.027573,-0.055617,0.185502,0.112014,-0.018345,0.031028,0.032948,0.024567,0.017578,0.024302,0.031580,0.067175,0.032872,0.054973,0.051084
P101,0.256764,-0.606593,0.266683,1.000000,0.013955,-0.199244,0.120240,0.992140,0.986280,0.992825,0.621157,0.533549,0.539609,-0.269606,0.026600,0.494589,0.060747,-0.005916,0.054651,0.491455,-0.019258,0.150265,0.069584,0.161638,0.068582,0.068411,-0.037984,0.166604,0.031721,-0.059783,0.069513,0.069994,0.064869,0.059683,0.064258,0.069068,0.068130,0.069575,0.052124,0.048276
AIT201,0.015445,-0.050169,0.015493,0.013955,1.000000,-0.028304,-0.026083,0.018081,0.009144,0.013432,0.006045,0.032588,0.022863,-0.053099,0.001178,0.019365,0.003879,0.003702,0.093513,-0.000625,0.124923,-0.301355,0.039669,-0.082884,0.070042,0.069827,0.164524,-0.096736,0.673429,-0.054363,0.039187,0.023333,0.087763,0.080933,0.064517,0.106721,0.193274,0.110224,0.003060,0.003089


# Plots

In [ ]:
# Box Plot
# We need to decide which values box plots are appropriate for. Numerical data with sufficient variation (not a binary 1 or 0)
import matplotlib.pyplot as plt
fig1, ax1 = plt.subplots()
ax1.set_title('Tank Levels ')
ax1.boxplot([sensor_data.LIT101,sensor_data.LIT301, sensor_data.LIT401])
ax1.set_xticklabels(['LIT101','LIT301','LIT401'])
plt.show(fig1)


In [ ]:
# Plot data for visual representation
# Note: The quantity of data means it takes a very long time to run. We might want to get a larger instance before doing heavy plotting.
import matplotlib.pyplot as plt
plt.plot(sensor_data.Timestamp.loc[1:1500],sensor_data.P201.loc[1:1500])
plt.show()

In [ ]:
sensor_data.head()